In [1]:
from dask.distributed import Client
dask_client = Client()
dask_client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51980,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:52004,Total threads: 2
Dashboard: http://127.0.0.1:52005/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:51984,


In [2]:
import getpass

In [89]:
cid = getpass.getpass(prompt="client_id for box")

client_id for box········


In [90]:
cid

'l8viwaotvjn2b5aqeyrm18az8a5gy9zx'

In [91]:
cs = getpass.getpass(prompt="client_secret for box")

client_secret for box········


In [92]:
cs

'S1oil2CDFcCAAPmvgXNdR3dMiSEU0U74'

In [93]:
atk = getpass.getpass(prompt="access token for box")

access token for box········


In [94]:
atk

'Ztl5ztmaZkZLxnxHs6zkjzOzPQZjWnQw'

In [95]:
from boxsdk import OAuth2, Client

In [96]:
auth = OAuth2(client_id=cid, client_secret=cs, access_token=atk)

In [97]:
client = Client(auth)

In [98]:
client.auth

In [99]:
file_id = '1278387018073'
file_content = client.file(file_id).content()

In [100]:
import io
file_id = '1281263195847'
file_content = client.file(file_id).content()
downloaded_data =  io.BytesIO(file_content)

In [101]:
downloaded_data.getbuffer()

In [102]:
# Write the stuff
with open("downloaded_file.csv", "wb") as f:
    f.write(downloaded_data.getbuffer())

In [103]:
import dask.dataframe as dd
ddf = dd.read_csv("downloaded_file.csv")
ddf

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,sys_created_by,sys_created_at,sys_updated_by,sys_updated_at,contact_type,location,category,subcategory,u_symptom,cmdb_ci,impact,urgency,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,bool,int64,int64,int64,bool,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,bool,bool,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [105]:
ddf_c = ddf.loc[ddf["incident_state"] == "Closed"].compute().copy()

In [106]:
del ddf

In [107]:
ddf_c

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
12,INC0000047,Closed,False,1,0,8,True,Caller 2403,Opened by 397,29/2/2016 04:40,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 81,1/3/2016 09:52,6/3/2016 10:00
19,INC0000057,Closed,False,0,0,6,True,Caller 4416,Opened by 8,29/2/2016 06:10,...,False,Do Not Notify,Problem ID 2,?,?,?,code 10,Resolved by 5,1/3/2016 02:55,6/3/2016 03:00
23,INC0000060,Closed,False,0,0,3,True,Caller 4491,Opened by 180,29/2/2016 06:38,...,False,Do Not Notify,?,?,?,?,code 3,Resolved by 113,2/3/2016 12:06,7/3/2016 13:00
31,INC0000062,Closed,False,1,0,7,False,Caller 3765,Opened by 180,29/2/2016 06:58,...,False,Do Not Notify,?,?,?,?,code 7,Resolved by 62,29/2/2016 15:51,5/3/2016 16:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141695,INC0120304,Closed,False,0,0,2,True,Caller 90,Opened by 8,15/2/2017 02:02,...,True,Do Not Notify,?,?,?,?,code 6,Resolved by 5,17/2/2017 00:47,17/2/2017 00:50
141697,INC0120319,Closed,False,0,0,1,True,Caller 1899,Opened by 508,15/2/2017 07:09,...,False,Do Not Notify,?,?,?,?,code 6,?,?,15/2/2017 07:09
141703,INC0120495,Closed,False,1,0,5,True,Caller 323,Opened by 14,15/2/2017 11:58,...,False,Do Not Notify,?,?,?,?,?,?,?,16/2/2017 09:51
141707,INC0120835,Closed,False,1,0,4,True,Caller 116,Opened by 12,16/2/2017 09:09,...,True,Do Not Notify,?,?,?,?,code 9,Resolved by 9,16/2/2017 09:53,16/2/2017 09:53


In [108]:
TIMESTAMP_COLS = ["sys_updated_at", "sys_created_at"]
ACTIVITY_COLS = ["assignment_group"]
COLS_NEEDED = ACTIVITY_COLS + TIMESTAMP_COLS

In [109]:
ddf_c = ddf_c[COLS_NEEDED]

In [111]:
df_complete = ddf_c[~((ddf_c["sys_updated_at"] == "?") \
                     | (ddf_c["sys_created_at"] == "?") | (ddf_c["assignment_group"] == "?"))].copy()

In [112]:
del ddf_c

In [113]:
import os
os.remove("downloaded_file.csv")

In [114]:
df_complete["assignment_group"].value_counts()

Group 25    1244
Group 39    1202
Group 24    1058
Group 23     812
Group 64     715
Group 28     544
Group 27     519
Group 20     395
Group 10     346
Group 57     311
Group 55     292
Group 30     268
Group 6      262
Group 29     257
Group 22     240
Group 54     201
Group 33     201
Group 31     198
Group 37     177
Group 56     168
Group 46     166
Group 48     162
Group 58     141
Group 5      140
Group 12     123
Group 65     111
Group 49     108
Group 3       98
Group 9       96
Group 53      91
Group 21      58
Group 15      58
Group 50      51
Group 34      51
Group 47      45
Group 59      43
Group 62      43
Group 51      38
Group 61      38
Group 19      37
Group 13      36
Group 60      24
Group 26      24
Group 35      17
Group 14      13
Group 17      11
Group 45      11
Group 44       9
Group 43       8
Group 32       8
Group 2        7
Group 63       7
Group 70       4
Group 18       2
Group 36       2
Group 73       2
Group 66       1
Group 41       1
Group 7       

In [116]:

df_complete["assignment_group"] = df_complete["assignment_group"].astype(str)
df_complete["sys_created_at"] = df_complete["sys_created_at"].astype("datetime64[ns]")
df_complete["sys_updated_at"] =  df_complete["sys_updated_at"].astype("datetime64[ns]")

In [117]:
df_complete.dtypes

assignment_group            object
sys_updated_at      datetime64[ns]
sys_created_at      datetime64[ns]
dtype: object

In [119]:
df_Q1 = df_complete[df_complete["sys_updated_at"].dt.quarter == 1]

In [120]:
del df_complete

In [121]:
df_Q1

,assignment_group,sys_updated_at,sys_created_at
84,Group 29,2016-03-24 18:40:00,2016-02-29 08:38:00
219,Group 23,2016-03-24 18:59:00,2016-02-29 09:13:00
314,Group 39,2016-03-24 18:50:00,2016-02-29 09:40:00
327,Group 24,2016-03-27 13:59:00,2016-02-29 09:48:00
388,Group 25,2016-03-19 10:00:00,2016-02-29 10:08:00
...,...,...,...
141260,Group 29,2017-01-24 11:00:00,2017-01-17 12:36:00
141300,Group 15,2017-02-14 17:00:00,2017-01-19 13:39:00
141310,Group 28,2017-01-28 11:00:00,2017-01-21 10:34:00
141325,Group 10,2017-01-29 17:00:00,2017-01-24 10:22:00


In [124]:
df_Q1["resolution_time"] = (df_Q1.loc[:, "sys_updated_at"] - df_Q1.loc[:, "sys_created_at"])

df_Q1["resolution_time"] = df_Q1["resolution_time"].dt.total_seconds()/(3600*24)

/var/folders/fb/t_m5qpcj6qq85rvkh73vbxh40000gn/T/ipykernel_3432/888407786.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Q1["resolution_time"] = (df_Q1.loc[:, "sys_updated_at"] - df_Q1.loc[:, "sys_created_at"])
/var/folders/fb/t_m5qpcj6qq85rvkh73vbxh40000gn/T/ipykernel_3432/888407786.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Q1["resolution_time"] = df_Q1["resolution_time"].dt.total_seconds()/(3600*24)


In [125]:
df_Q1[df_Q1["resolution_time"] < 0]

,assignment_group,sys_updated_at,sys_created_at,resolution_time
495,Group 54,2016-02-03 13:02:00,2016-02-29 10:52:00,-25.909722
1100,Group 31,2016-02-04 14:59:00,2016-02-29 15:50:00,-25.035417
4498,Group 5,2016-01-05 15:07:00,2016-02-03 10:42:00,-28.815972
4869,Group 10,2016-01-05 12:07:00,2016-02-03 11:38:00,-28.979861
5507,Group 10,2016-01-05 14:07:00,2016-02-03 13:55:00,-28.991667
...,...,...,...,...
141158,Group 9,2017-02-13 15:00:00,2017-11-01 09:49:00,-260.784028
141172,Group 12,2017-01-18 10:00:00,2017-11-01 11:45:00,-287.072917
141176,Group 48,2017-01-16 14:00:00,2017-11-01 11:46:00,-288.906944
141195,Group 55,2017-01-17 18:00:00,2017-11-01 17:04:00,-287.961111


In [83]:
df_Q1.to_csv("Q1_ticket_activity_summary.csv", index=False)

In [59]:
folder_id = '221826297443'
new_file = client.folder(folder_id).upload("Q1_ticket_activity_summary.csv")

In [60]:
os.remove("Q1_ticket_activity_summary.csv")

In [64]:
df_Q1["resolution_time"]

83        24.418056
216       24.406944
218       24.406944
313       24.381944
326       27.174306
            ...    
141259     6.933333
141299    26.139583
141309     7.018056
141324     5.276389
141387    18.832639
Name: resolution_time, Length: 3624, dtype: float64